In [ ]:
import numpy as np
import pyscf
import py3Dmol
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
from oneqmc.analysis.visual import show_mol
from oneqmc.convert_geo import load_molecules
from oneqmc.analysis.plot import set_defaults
from oneqmc.analysis import HARTREE_TO_KCAL, colours

set_defaults()

In [ ]:
CREATE_FIGURES = False

In [ ]:
finetune_params = np.load(
    "../../experiment_results/05_alkane_scalability/finetune_params.npz"
)

In [ ]:
dataset = load_molecules(
    "../../data/alkanes/scalability", json_whitelist="alkane(6|7|8|9|10|11|12|13)"
)

#### Isosurfaces of envelopes

In [ ]:
class CubeDataFormatter:
    def __init__(self, mol, nx=50, ny=50, nz=50):
        self.charges = mol.charges
        self.coords = mol.coords
        margin = 3.0
        extent = np.max(mol.coords, axis=0) - np.min(mol.coords, axis=0) + 2 * margin
        self.box = np.diag(extent)
        self.boxorig = np.min(mol.coords, axis=0) - margin

        self.nx = nx
        self.ny = ny
        self.nz = nz
        self.xs = np.linspace(0, 1, nx)
        self.ys = np.linspace(0, 1, ny)
        self.zs = np.linspace(0, 1, nz)

    def get_coords(self):
        frac_coords = np.stack(np.meshgrid(self.xs, self.ys, self.zs), axis=-1)
        # permuting x<->y is necessary to match weird ordering of cube format
        return np.einsum("yxzi,ij->xyzj", frac_coords, self.box) + self.boxorig

    def __call__(self, field) -> str:
        assert field.ndim == 3
        assert field.shape == (self.nx, self.ny, self.nz)
        comment = ""
        string = ""
        string += comment + "\n"
        string += "Created by OneQMC CubeFormatter\n"
        string += f"{len(self.coords):5d}"
        string += "{:12.6f}{:12.6f}{:12.6f}\n".format(*tuple((self.boxorig).tolist()))
        dx = self.xs[-1] if len(self.xs) == 1 else self.xs[1]
        dy = self.ys[-1] if len(self.ys) == 1 else self.ys[1]
        dz = self.zs[-1] if len(self.zs) == 1 else self.zs[1]
        delta = (self.box.T * np.stack([dx, dy, dz])).T
        string += (
            f"{self.nx:5d}{delta[0,0]:12.6f}{delta[0,1]:12.6f}{delta[0,2]:12.6f}\n"
        )
        string += (
            f"{self.ny:5d}{delta[1,0]:12.6f}{delta[1,1]:12.6f}{delta[1,2]:12.6f}\n"
        )
        string += (
            f"{self.nz:5d}{delta[2,0]:12.6f}{delta[2,1]:12.6f}{delta[2,2]:12.6f}\n"
        )
        for charge, coord in zip(self.charges, self.coords):
            string += "%5d%12.6f" % (charge, 0.0)
            string += "{:12.6f}{:12.6f}{:12.6f}\n".format(*tuple((coord).tolist()))

        # Sync to CPU if not there already
        field = np.asarray(field)
        for ix in range(self.nx):
            for iy in range(self.ny):
                for iz0, iz1 in pyscf.lib.prange(0, self.nz, 6):
                    fmt = "%13.5E" * (iz1 - iz0) + "\n"
                    string += fmt % tuple(field[ix, iy, iz0:iz1].tolist())

        return string


def show_isosurface(
    field_data,
    iso_value: float = 0.05,
    view=None,
):
    if view is None:
        view = py3Dmol.view()
    view.addVolumetricData(
        field_data,
        "cube",
        {
            "isoval": -iso_value,
            "smoothness": 5,
            "opacity": 0.8,
            "volformat": "cube",
            "color": "blue",
        },
    )
    view.addVolumetricData(
        field_data,
        "cube",
        {
            "isoval": iso_value,
            "smoothness": 5,
            "opacity": 0.8,
            "volformat": "cube",
            "color": "red",
        },
    )
    return view

In [ ]:
orb_ix = 35  # delocalized over many carbons
mol_ix = 2

In [ ]:
formatter = CubeDataFormatter(dataset[mol_ix], nx=150, nz=150)

In [ ]:
grid = formatter.get_coords()

In [ ]:
def envelope_fn(x, exponents, centers, coefs):
    r = np.linalg.norm(centers - x[..., None, :], axis=-1)
    exps = np.exp(-exponents * r[..., None])
    return np.einsum("...ij,ij->...", exps, coefs)

In [ ]:
coef = finetune_params[f"envs_lac_{mol_ix}_se_envelope_up_feature_selector"][
    orb_ix, :, :, 0
]
field_values = envelope_fn(
    grid,
    finetune_params[f"envs_lac_{mol_ix}_exponents"].squeeze(0),
    dataset[mol_ix].coords,
    coef,
)
# Easier to visualize if it integrates to 1
field_values /= np.abs(field_values).sum()

In [ ]:
field_data = formatter(field_values)

In [ ]:
view = show_mol(dataset[mol_ix])
show_isosurface(field_data, view=view, iso_value=1e-5)
view.rotate(90, 'x')

### Seeing the overall sparsity pattern

In [ ]:
def localization_score(name, mol_ix):
    coef_up = finetune_params[f"envs_{name}_{mol_ix}_se_envelope_up_feature_selector"][
        :, :, :, 0
    ]
    coef_down = finetune_params[
        f"envs_{name}_{mol_ix}_se_envelope_down_feature_selector"
    ][:, :, :, 0]
    c = np.concatenate([coef_up, coef_down], axis=-1)
    abs_c = np.abs(c).sum(-1)
    loc_score = abs_c.max(-1) / abs_c.sum(-1)
    return loc_score

In [ ]:
plt.figure(figsize=(8.5, 3.5))
plt.plot(
    localization_score("lac", mol_ix), label="Finetuned from LAC", c=colours.DARKTEAL
)
plt.plot(localization_score("scratch", mol_ix), label="From scratch", c=colours.TEAL)
plt.plot(
    localization_score("final_pretrain", mol_ix),
    label="LAC pretrained",
    c=colours.DARKYELLOW,
)
plt.plot(
    localization_score("10k_pretrain", mol_ix),
    label="LAC pretrained 10k",
    c=colours.YELLOW,
)

plt.xlabel("Orbital index")
plt.ylabel("Localization score")
plt.legend(fontsize=10)

In [ ]:
slaters = np.load("../../experiment_results/05_alkane_scalability/slaters.npz")

In [ ]:
s8 = slaters["slaters_lac_2"][0]
s7 = np.zeros_like(s8)
mask = np.ones_like(s8)
s7[:58, :42] = slaters["slaters_lac_1"][0][:, :42]
s7[:58, 48:64] = slaters["slaters_lac_1"][0][:, 42:]
mask[:58, :42] = 0
mask[:58, 48:64] = 0

In [ ]:
vmax = max(np.max(np.abs(s8 - s7)), np.max(np.abs(s8)), np.max(np.abs(s7)))

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

cmap0 = LinearSegmentedColormap.from_list("", ["white", "darkblue"])
cmap1 = LinearSegmentedColormap.from_list("", ["white", "lightgrey"])

In [ ]:
plt.imshow(np.abs(s8 - s7), vmin=0, vmax=vmax, cmap=cmap0)
plt.imshow(np.ones_like(s7), alpha=mask.astype("float"), cmap=cmap1)
plt.ylabel("Electrons ordered by $x$ coord")
plt.annotate("Electrons non-existent in heptane", (12, 62.5), fontsize=10)
plt.annotate("Orbitals non-existent in heptane", (44, 50), fontsize=10, rotation=90)
plt.colorbar()
plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.show()

In [ ]:
plt.imshow(np.abs(s8), vmin=0, vmax=vmax, cmap=cmap0)
plt.ylabel("Electrons ordered by $x$ coord")
plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.show()

In [ ]:
plt.imshow(np.abs(slaters["slaters_lac_1"][0]), vmin=0, vmax=vmax, cmap=cmap0)
plt.ylabel("Electrons ordered by $x$ coord")
plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.show()

In [ ]:
plt.imshow(np.abs(slaters["grads_lac_2"][0]), vmin=0, cmap=cmap0)
plt.ylabel("Electrons ordered by $x$ coord")
plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.show()

# Linear scaling

In [ ]:
df = pd.read_csv("../../experiment_results/05_alkane_scalability/energy.csv")

In [ ]:
def plot_linearly_extrapolated(subdf, ax):
    predictors = np.stack([subdf["C"], np.ones_like(subdf["C"])], axis=-1)
    targets = HARTREE_TO_KCAL * subdf["rmean"]
    output = np.linalg.lstsq(predictors, targets, rcond=None)
    fitted_values = predictors @ output[0]
    residuals = targets - fitted_values
    mae = (residuals**2).mean()

    ax.plot(subdf["C"], residuals, color=colours.DARKERTEAL)
    ax.set_xlabel("C")
    ax.set_ylabel("Residual from linear fit (kcal/mol)")
    ax.text(7, 0.8, f"MSE={mae:.3}", fontsize=15, c=colours.DARKERTEAL)
    ax.axhspan(-1, 1, color=colours.TEAL, alpha=0.25, lw=0)

In [ ]:
plot_linearly_extrapolated(
    df[(df["sampler"] == "double-langevin") & (df["steps"] == "180000")], plt.gca()
)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))
plot_linearly_extrapolated(
    df[(df["sampler"] == "double-langevin") & (df["steps"] == "180000")], axs[0]
)

subdf1 = df[(df["sampler"] == "double-langevin") & (df["steps"] == "180000")]
subdf2 = df[df["mode"] == "CCSD(T)"]

x = subdf1["C"]
y = HARTREE_TO_KCAL * (np.array(subdf1["rmean"]) - np.array(subdf2["rmean"]))

coef = np.polyfit(x, y, 1)
poly1d_fn = np.poly1d(coef)
axs[1].set_xlabel("C")
axs[1].set_ylabel("Orbformer - CCSD(T)/CBS (kcal/mol)")

axs[1].plot(x, y, "o", x, poly1d_fn(x), "-", c=colours.TEAL)
fig.tight_layout()
if CREATE_FIGURES:
    plt.savefig("linear-scaling-two-plot.pdf", dpi=600)